In [ ]:
import os
import json
import requests
from dotenv import load_dotenv

# we now use OpenAI for convenience methods
from openai import OpenAI

In [24]:
from rich.console import Console

console = Console()

In [ ]:
# We are familiar with a standard API request
# This is a public API that returns a random joke
# The API developers have a very deterministic response even if random
# Both sides program imperatively

get_random_joke_internet = requests.get(
    "https://official-joke-api.appspot.com/random_joke"
)

print(get_random_joke_internet.json())

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins:{openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

In [ ]:
client = OpenAI()
MODEL = "gpt-4o-mini"
print(f"MODEL: {MODEL}")

In [28]:
# original propmt
system_message = """
You are an assistant that is great at telling jokes.
"""

# Here is where we can do some prompt engineering - we are adding to the system message
prompt_engineering = """
A joke worthy of publishing is a joke that has a rating of 8.5/10 or above.

If the joke is worthy of publishing also include next: PUBLISH otherwise next: RETRY

Here is an example of a joke worth of publishing:
Supply the respoinse in the following format:
|{"setup": "The setup of the joke",
"punchline": "The punchline of the joke",   
"rating": "9.0",
"next": "PUBLISH"
}|

Remove all back ticks and other unnecessary characters and just print the JSON format and nothing else.

Please ensure jokes are not repeated on retries

Thank you.

"""

system_message += prompt_engineering

user_prompt = "Tell a light-hearted joke for an audience of Pythonistas"

In [29]:
# We create a list of messages to pass to the LLM

prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt},
]

In [ ]:
# We saw previously what is returned in the response object

response = client.chat.completions.create(model=MODEL, messages=prompts)

print(response.choices[0].message.content)

output = response.choices[0].message.content.replace("\n", "")

For the application, we create a state object so that we can track the state of the application - it is custom.


In [31]:
state = {"next": "", "setup": "", "punchline": "", "rating": ""}

In [ ]:
# We split the output to extract the JSON - there are better ways of getting structured output but this is for demo purposes

output_json = output.split("|")
output_dict = json.loads(output_json[1])
print(type(output_dict))

console.print(output_dict)

# We update the state object and the app will have information on the NEXT step...

if output_dict["next"] == "PUBLISH":
    state["next"] = output_dict["next"]
    state["setup"] = output_dict["setup"]
    state["rating"] = output_dict["rating"]
    state["punchline"] = output_dict["punchline"]

else:
    state["next"] = "RETRY"

In [ ]:
# We can extract the NEXT step from the Autonomous AI Agent

console.print(f"next: [dark_orange bold]{state['next']}[/]")